In [2]:
import pandas as pd

df = pd.read_csv("라벨.csv", encoding='cp949')
df

,row,표제어,단어1,단어2,단어3,단어4,단어5,단어6,단어7,단어8,단어9,단어10,단어11,단어12
0,2,가건물,temporary building,makeshift building,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,가계부,household account book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,가구점,furniture store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,가드,guard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,가락국수,garakguksu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761,1763,흑설탕,raw sugar,unrefined sugar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1762,1764,흑판,blackboard,chalkboard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1763,1765,흙담,adobe wall,mud wall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1764,1766,흙집,mud house,house of earth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# 표제어 리스트
query_list = list(df['표제어'])

# 클래스 리스트
class_list = []
for i in range(len(df)):
    class_list.append(list(df.iloc[i][2:].dropna()))

In [4]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO

# CLIP 모델과 프로세서 로드
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# 이미지 경로와 텍스트 레이블 목록을 받아서, 이미지에 가장 적합한 텍스트 레이블을 예측합니다.
def predict_text_from_image(image_url, text_labels):
    # 이미지 로드 및 전처리
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    inputs = processor(text=text_labels, images=image, return_tensors="pt", padding=True)

    # 모델을 통해 이미지와 텍스트의 유사도 계산
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image # 이미지에 대한 로짓

    # 수정 후
    probs = logits_per_image.softmax(dim=1).detach().cpu().numpy()

    # 가장 높은 확률을 가진 텍스트 레이블을 찾음
    # max_index = probs.argmax()
    # predicted_label = text_labels[max_index]

    return probs[0].tolist()

c:\Users\uilov\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
import requests
from bs4 import BeautifulSoup

# 이미지 크롤링 + 모델 결과 json화
def crawl(query, labels) :
    # 크롤링 초기화
    crawl_list = []
    
    # 검색
    url = 'https://www.google.com/search?sca_esv=2e034f13424ae33d&q=' + query + '&tbm=isch&source=lnms&sa=X&ved=2ahUKEwjTlIyLu8CEAxUfjVYBHVNEARoQ0pQJegQIDhAB&biw=1600&bih=927&dpr=1.56'
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    
        for jdx, j in enumerate(soup.find_all('img')) :
            if jdx == 0 :
                continue
            # 모델 결과
            probs = predict_text_from_image(j['src'], labels)
            
            # json_list화
            crawl_list.append({'url' : j['src'], 'probs' : probs})
            
        return crawl_list

In [6]:
# result를 받아 업데이트하는 구조
def make_dict(result:list, index:int, query:str, labels:list):
    crawl_list = crawl(query, labels)

    for i in crawl_list :
        url = i['url'] # url
        probs = i['probs'] # probs
        result.append({'query' : query, 'url' : url, 'labels' : labels, 'probs': probs}) # json_list
    return result

In [7]:
# 빈 데이터셋 만들기
df2 = pd.DataFrame(data=[], columns=["index", "url", "표제어", "단어1", "단어2", "단어3", "단어4", "단어5", "단어6", "단어7", "단어8", "단어9", "단어10", "단어11", "단어12", "확률1", "확률2", "확률3", "확률4", "확률5", "확률6", "확률7", "확률8", "확률9", "확률10", "확률11", "확률12"])
df2

,index,url,표제어,단어1,단어2,단어3,단어4,단어5,단어6,단어7,...,확률3,확률4,확률5,확률6,확률7,확률8,확률9,확률10,확률11,확률12


In [8]:
y = 0
all_number = len(query_list)

for idx, i in enumerate(class_list) :
    ## 부분 작동 시
    # if idx > 627 :
    #     break
    
    # 클래스가 1개 이상의 경우에만
    if len(i) > 1 :
        result = []
        try :
            # json_list받기
            result = make_dict(result, idx+2, query_list[idx], i)
            
            # dictionary형태로 변환 후 df2에 기입
            for j in result :
                dict = {}
                dict["index"] = idx+2
                dict["url"] = j["url"]
                dict["표제어"] = j['query']
                

                for x_idx, x in enumerate(j['labels']) :
                    dict_key1 = "단어" + str(int(x_idx + 1))
                    dict_key2 = "확률" + str(int(x_idx + 1))

                    dict[dict_key1] = x
                    dict[dict_key2] = j['probs'][x_idx]
            
                df2.loc[y] = pd.Series(dict)
                y += 1
        except:
            print(f"{i}는 오류")

    # csv로 중간 저장
    df2.to_csv("결과.csv", encoding='cp949')            
    print(f"진행 상황 : {round(((idx+1)/all_number)*100,1)}%")

진행 상황 : 0.1%
진행 상황 : 0.1%
진행 상황 : 0.2%
진행 상황 : 0.2%
진행 상황 : 0.3%
진행 상황 : 0.3%
['crossbeam', 'crossbar', 'rail']는 오류
진행 상황 : 0.4%
진행 상황 : 0.5%
진행 상황 : 0.5%
['writing from left to right', 'writing laterally']는 오류
진행 상황 : 0.6%
진행 상황 : 0.6%
진행 상황 : 0.7%
진행 상황 : 0.7%
진행 상황 : 0.8%
진행 상황 : 0.8%
진행 상황 : 0.9%
